# Required Libraries

To start, we will be importing several libraries:


1.   Tensorflow - A commonly used library in artificial intelligence and machine learning. Includes several classes and functions that are crucial in building models.
2.   Keras - A library that provides an interface for Tensorflow. Simplifies building neural networks since it is built in Python.
3.   Numpy - A library that supports computational operations related to linear algebra, matrices, and fourier transform.
6.   os - A library which allows us to interact with the operating system (files, directories, etc.)



In [ ]:
import tensorflow as tf
import numpy as np
import random
import os

#test function to make sure our gpu is working
tf.test.gpu_device_name()

'/device:GPU:0'

# Taking a Look at the Data

Our training data is currently stored in a text file so we begin by opening and reading the file.

To make sure our data is read properly, we print out a small chunk of the file.

In [ ]:
text = open("/content/text.txt", "r").read()
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


We'll need to train the model to predict how Shakespeare puts together words so we'll also need to know how many different characters there are first (65 in this case).

In [ ]:
vocab = sorted(set(text))
print("Number of Unique Characters: {}".format(len(vocab)))

Number of Unique Characters: 65


# Processing the Text

**Encoding and Decoding the characters**

Before we even begin training the model, we need to process our data into something that the model can actually read and understand. In this case, we will be converting each character in the file into a numerical representation.

**Mapping Each Char to an Index**

We'll assign a number to each character in the vocabulary set using enumerate() and place the pairs in a dictionary. After we've done this, we'll go ahead and convert the entire text file into numbers and store it into a numpy array.

**Mapping Each Index to a Char**

Next, we'll store the entire list of characters as a numpy array as well so that we'll be able to use each character as a key in the dictionary when decoding.


In [ ]:
char2Index = {c : i for i, c in enumerate(vocab)}
int_text = np.array([char2Index[i] for i in text])

index2Char = np.array(vocab)

To check that our conversions worked properly, we will print the dictionary and chunks of each new array to compare to each other.

We use repr() to allow us to print out representations of each character so that it is easier to visualize.

In [ ]:
print("Character to Index: \n")
for i in char2Index:
  print("  {:4s}: {:3d}".format(repr(i), char2Index[i]))

print("\nInput text to Integer: \n")
print('{} mapped to {}'.format(repr(text[:20]), int_text[:20]))

Character to Index: 

  '\n':   0
  ' ' :   1
  '!' :   2
  '$' :   3
  '&' :   4
  "'" :   5
  ',' :   6
  '-' :   7
  '.' :   8
  '3' :   9
  ':' :  10
  ';' :  11
  '?' :  12
  'A' :  13
  'B' :  14
  'C' :  15
  'D' :  16
  'E' :  17
  'F' :  18
  'G' :  19
  'H' :  20
  'I' :  21
  'J' :  22
  'K' :  23
  'L' :  24
  'M' :  25
  'N' :  26
  'O' :  27
  'P' :  28
  'Q' :  29
  'R' :  30
  'S' :  31
  'T' :  32
  'U' :  33
  'V' :  34
  'W' :  35
  'X' :  36
  'Y' :  37
  'Z' :  38
  'a' :  39
  'b' :  40
  'c' :  41
  'd' :  42
  'e' :  43
  'f' :  44
  'g' :  45
  'h' :  46
  'i' :  47
  'j' :  48
  'k' :  49
  'l' :  50
  'm' :  51
  'n' :  52
  'o' :  53
  'p' :  54
  'q' :  55
  'r' :  56
  's' :  57
  't' :  58
  'u' :  59
  'v' :  60
  'w' :  61
  'x' :  62
  'y' :  63
  'z' :  64

Input text to Integer: 

'First Citizen:\nBefor' mapped to [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


# Training Data

The next step for us is to now split our data into sequences. Our model is going to be fed these sequences and it will, over time, learn to predict the characters that follow each sequence. This will allow it to generate text with Shakespearean mannerisms.

We'll begin by setting the length of each sequence. We will set it to 150 characters here. Then, by using from_tensor_slices(), we can convert the text from vector form into a character index stream which we can split up into batches.

In [ ]:
seq_length = 150  # max number of characters that can be fed per input

char_dataset = tf.data.Dataset.from_tensor_slices(int_text)

By calling the .batch() function and setting the first parameter to the sequence length, we will be splitting the dataset into batches each with 150 consecutive characters. The drop_remainder parameter is set to True to ensure that each batch has the same outer dimension.

In [ ]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

From here, we'll want to create tuples of these sequences that can actually be fed to our model. The reason why we need tuples is because for RNN models to work, there needs to be an input text and target text for it to output. The input text is the sequence itself and the target text is the sequence minus its first character but with an additional predicted character at the end.

By calling map(create_input_target_pair) we will be applying the function to each element of our dataset.

In [ ]:
def create_input_target_pair(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(create_input_target_pair)

Finally, we will shuffle the data before packing it into the final batches which will be fed into the model for training. Shuffling the data set allows for the training set to be more representative of the entire text that we have since it minimizes variance.

In [ ]:
BATCH_SIZE = 64

# Buffer used for shuffling dataset
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 150), dtype=tf.int64, name=None), TensorSpec(shape=(64, 150), dtype=tf.int64, name=None))>


# Building the Model

To begin building the model, we first set parameters that we will be feeding into the layers of the model.

In [ ]:
vocab_size = len(vocab)
# Embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units= 1024

Our model will be able to take in 64 sequences per batch as of now, however, we will want to be able to input single sequences after training the model so we'll need to be able to use a different batch size later. To do this, we will create a function that can reproduce the model with different batch sizes.

We will be using what's called a character-level recurrent neural network for this workshop. Recurrent neural networks are generally used for predictive operations on sequential data. It works by incorporating memory to save information from prior inputs and previously seen elements as "states" to influence the current output. The reason why our RNN is classified as character-level is because it reads words as a series of characters.
The model itself will consist of 3 layers:
1. Input Layer - This layer uses an embedding layer to map each character-ID to a 256 dimension vector.
2. GRU Layer - This layer is a type of recurrent neural network with a size of 1024 gradient descent units.
3. Dense Layer - This is the output layer which is the same size as the size of vocab. It essentially outputs the probability of each character being the next one in the sequence.

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model


In [ ]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In this model, we will once again be using the adam learning algorithm and the sparse categorical crossenttropy loss function. We are setting the from_logits parameter to True to inform the loss function that the output values generated by the model are not normalized.

In [ ]:
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

To be able to load our weights and to save our training performance, we will need to configure a checkpoint directory. We'll be using the saved checkpoint when we reproduce the model with a different batch size.

In [ ]:
dir_checkpoints = './training_checkpoints'
checkpoint_prefix = os.path.join(dir_checkpoints, "checkpoint_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

Before training the model, the last thing we need to do is set the number of epochs. In this specific workshop, more epochs will lead to less loss, however, in the interest of time, we will only be using 10 epochs. Feel free to modify this number and see how your model differs later on though.

In [ ]:
EPOCHS=10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
115/115 [==============================] - 19s 84ms/step - loss: 2.9145
Epoch 2/10
115/115 [==============================] - 11s 83ms/step - loss: 2.1567
Epoch 3/10
115/115 [==============================] - 11s 83ms/step - loss: 1.8870
Epoch 4/10
115/115 [==============================] - 11s 85ms/step - loss: 1.6989
Epoch 5/10
115/115 [==============================] - 12s 89ms/step - loss: 1.5747
Epoch 6/10
115/115 [==============================] - 12s 93ms/step - loss: 1.4877
Epoch 7/10
115/115 [==============================] - 13s 96ms/step - loss: 1.4265
Epoch 8/10
115/115 [==============================] - 13s 96ms/step - loss: 1.3777
Epoch 9/10
115/115 [==============================] - 12s 93ms/step - loss: 1.3384
Epoch 10/10
115/115 [==============================] - 12s 87ms/step - loss: 1.3047


After training, we can view the location of our latest checkpoint.

In [ ]:
tf.train.latest_checkpoint(dir_checkpoints)

'./training_checkpoints/checkpoint_10'

Now, we'll go ahead and create a new model using the earlier function with a different batch size and weights which are loaded from our checkpoint.

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(dir_checkpoints))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Our model is now ready to make predictions, and all we need is a custom function to prepare our input for the model. We have to do the following:

1. Set the number of characters to generate
2. Vectorizing the input (from string to numbers) to feed into the model
3. Create an empty variable to store the result
4. Create a temperature value to manually adjust variability of the predictions
5. Feed the output to the model again for the next prediction
6. Join the generated character to the final string

In [ ]:
def generate_text(model, start_string):
    num_generate = 1000 #Number of characters to be generated

    input_eval = [char2Index[s] for s in start_string] # Converting input to indexes
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Reset any hidden states in the model
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a categorical distribution to get the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We set the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(index2Char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
test = input("Enter your starting string: ")
print(generate_text(model, start_string=test))

Enter your starting string: oh thow is deth?
oh thow is deth?

GLOUCESTER:
And so, sir, I know the cause of worthing doth
Be a senden for a man could not stame a little good
Show'd with the citizens, and his friends are no sounding to him and his words,
That should he can crave the grave. The land and thy head,
And sure the trumpets of Rome and her for him,
And thou the very weeds of the thrown the heavens and my heart
And stand up thy love, or my heart to them,
When they are none of the court of thy word.

KING RICHARD III:
That should consure the causes of men are well prove
And see him in a life being so did be cut one and rebuty.

ROMEO:
I am a godden for this face to come to make
A flier and a very grave be the city is dead.

PETRUCHIO:
Ay, but that thou shalt not stand upon a charge
Our such a cousin Claudio's death,
And threaten and consiners, when he did such a cruel the rettle of him.

PAULINA:
No more of the courtier of the mother reason
That should stand before the sun that 